In [ ]:
%%capture
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
log = logging.getLogger()
log.setLevel(logging.DEBUG)

import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
from hyperoptim.results import load_data_frame, remove_columns_with_same_value, dataset_averaged_frame
from hyperoptim.results import mean_identical_exps
import pandas as pd
from hyperoptim.results import remove_columns_with_same_value, remove_indices_with_same_value

In [ ]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
log = logging.getLogger()
log.setLevel(logging.DEBUG)

In [ ]:
from scipy.stats.morestats import wilcoxon
import random

def to_numeric_where_possible(df):
    df = df.copy(deep=True)
    for col in df.columns:
        df.loc[:,col] = pd.to_numeric(df.loc[:,col], errors='ignore')
    return df

def round_numeric_columns(df, decimals):
    df = df.copy(deep=True)
    tmp = df.select_dtypes(include=[np.number], exclude=[np.timedelta64])
    df.loc[:, tmp.columns] = np.round(tmp, decimals)
    return df

def perm_mean_diffs_sampled(a, b, n_diffs=None):
    """Compute differences between all permutations of  labels.
    Version that samples.
    Parameters
    --------------
    a: list or numpy array
    b: list or numpy array
    n_diffs: int
        How many diffs/samples to compute.
    Returns
    -------
    all_diffs: 1d-array of float
        Sampled mean differences.
    """
    
    n_exps = len(a)
    all_bit_masks = [2 ** n for n in range(n_exps-1,-1,-1)]
    if n_diffs is None:
        n_diffs = 2**n_exps
        i_all_masks = range(n_diffs)
    else:
        random.seed(39483948)
        # take samples of all masks, always add identity mask
        i_all_masks = random.sample(range(0,2**n_exps-1), n_diffs - 1)
        i_all_masks = i_all_masks + [(2**n_exps)-1]
        # verification this is actually identity mask for code below:
        test_i_mask = i_all_masks[-1]
        test_mask = (np.bitwise_and(test_i_mask, all_bit_masks) > 0) * 2 - 1
        assert np.array_equal(a - b, (test_mask * a)  -test_mask * b)

        
    all_diffs = np.float32(np.ones(n_diffs) * np.nan)
    for i_diff, i_mask in enumerate(i_all_masks):
        # masks has -1s and 1s,
        # 1 interpretable as
        # correct value selected
        # -1 as randomly flipped value/"incorrect" value selected
        # *2 makes values between 2 and 0, then -1 to make 
        # values between 1 and -1
        mask = (np.bitwise_and(i_mask, all_bit_masks) > 0) * 2 - 1
        # mean later by dividing by n_exp
        # seems to be a little bit faster that way
        diff = np.sum((mask * a)  -mask * b)
        all_diffs[i_diff] = diff
    all_diffs = all_diffs / float(n_exps)
    return all_diffs

def perm_mean_diffs(a,b):
    """Compute differences between all permutations of  labels.
    Assumes a and b are paired values,
    a are values with label 0 and b with label 1.
    Computes mean differences for all possible   
    switches of 0 and 1 (but keeping pairs together, i.e.
    2 ^ len(a) switches).
    
    Parameters
    --------------
    a: list or numpy array
    b: list or numpy array
    
    Returns
    -------
    diffs: 1d-numpy array
        Differences between means of labelled values
        for all label-switched values.
        
    Notes
    -----
    http://www.stat.ncsu.edu/people/lu/courses/ST505/Ch4.pdf#page=10
    http://stats.stackexchange.com/a/64215/56289
    http://www.jarrodmillman.com/publications/millman2015thesis.pdf ->
    https://github.com/statlab/permute python package 
    (probably, didnt read: http://finzi.psych.upenn.edu/R/library/EnvStats/html/twoSamplePermutationTestLocation.html)
    """
    a = np.array(a)
    b = np.array(b)
    assert len(a) == len(b)
    n_exps = len(a)
    all_masks = _create_masks(n_exps)
    diffs = _compute_diffs(a, b, all_masks)
    return diffs

def perm_mean_diff_test(a,b, n_diffs=None):
    """Return two sided p-value of perm mean diff."""
    if n_diffs is None:
        diffs = perm_mean_diffs(a, b)
    else:
        diffs = perm_mean_diffs_sampled(a, b, n_diffs)
    
    actual_diff = np.mean(a - b)
    n_samples_as_large_diff = np.sum(np.abs(diffs) >= np.abs(actual_diff))
    #if n_diffs is not None:
    #    p_val = n_samples_as_large_diff + 1 /
    return n_samples_as_large_diff / float(len(diffs))


def _create_masks(n_exps):
    """ Create all (2^n_exps) binary selection masks for this number of experiments.
    E.g. for 3 experiments:
    False, False, False
    False, False, True
    False, True, False
    False, True, True
    True, False, False
    True, False, True
    True, True, False
    True, True, True""" 
    all_masks = np.array([[False] * n_exps] * (2 ** n_exps))
    i_block_size = all_masks.shape[0] // 2 
    for i_col in range(0,all_masks.shape[1]):
        for i_row in range(0,all_masks.shape[0], i_block_size * 2):
            all_masks[i_row:i_row+i_block_size,i_col] = [[True]] * i_block_size
        i_block_size //= 2
    return all_masks
    
def _compute_diffs(a, b, all_masks):
    # first add "first set" part
    # positive labels from a
    # and negative labels from b
    diffs = all_masks * a
    diffs += (1 - all_masks) * b
    # subtract "second set" part
    # negative labels from a
    # positive labels from b
    diffs -= (1 - all_masks) * a
    diffs -= all_masks * b
    return np.mean(diffs, axis=1)

def pairwise_compare_frame(df, with_p_vals=False, result_cols=('test', 'time', 'train',
        'test_sample', 'train_sample'), compare_col='test'):
    table_vals = []
    table_indices = []
    param_keys = set(df.keys()) - set(list(result_cols))
    for key in param_keys:
        if key == 'dataset_filename' or key == 'test_filename' or key == 'subject_id':
            continue
        possible_vals = df[key].unique()
        for i_value_a in range(0, len(possible_vals) - 1):
            for i_value_b in range(i_value_a + 1, len(possible_vals)):
                val_a = possible_vals[i_value_a]
                val_b = possible_vals[i_value_b]
                frame_1 = df[df[key] == val_a]
                frame_2 = df[df[key] == val_b]
                other_param_keys = list(param_keys - set([key]))
                joined_frame = frame_1.merge(frame_2, on=other_param_keys)
                if joined_frame.size == 0:
                    continue
                accuracies_a = np.array(joined_frame[compare_col + '_x'],
                    dtype=np.float64)
                accuracies_b = np.array(joined_frame[compare_col + '_y'],
                    dtype=np.float64)
                mean_a = np.mean(accuracies_a)
                mean_b = np.mean(accuracies_b)
                # Always put better value first in table
                if mean_a >= mean_b:
                    accuracies_1 = accuracies_a
                    accuracies_2 = accuracies_b
                    mean_1 = mean_a 
                    mean_2 = mean_b 
                    val_1 = val_a
                    val_2 = val_b
                else:
                    accuracies_1 = accuracies_b
                    accuracies_2 = accuracies_a
                    mean_1 = mean_b 
                    mean_2 = mean_a 
                    val_1 = val_b
                    val_2 = val_a
                if with_p_vals:
                    if len(accuracies_1) <= 18:
                        diff_perm = perm_mean_diff_test(accuracies_1,
                            accuracies_2) * 100
                    elif len(accuracies_1) <= 62:
                        diff_perm = perm_mean_diff_test(accuracies_1,
                            accuracies_2, n_diffs=2**17) * 100
                    else:
                        _, diff_perm = wilcoxon(accuracies_1,
                            accuracies_2)
                        diff_perm *= 100

                diffs = accuracies_2 - accuracies_1
                diff_std = np.std(diffs)
                diff_mean = np.mean(diffs)
                this_vals = [len(accuracies_1), str(val_1), str(val_2),
                    mean_1, mean_2, diff_mean, diff_std]
                if with_p_vals:
                    this_vals.append(diff_perm)
                table_vals.append(this_vals)
                table_indices.append(key)

    if len(table_vals) == 0:
        return None
    table_vals = np.array(table_vals)
    compare_headers = ['n_exp', 'val_1', 'val_2', 'acc_1', 'acc_2',
                       'diff', 'std']
    if with_p_vals:
        compare_headers.append('p_val')
    compare_frame = pd.DataFrame(table_vals, columns=compare_headers,  
                                 index=(table_indices))
    compare_frame = to_numeric_where_possible(compare_frame)
    compare_frame = round_numeric_columns(compare_frame, 1)
    return compare_frame

result_cols = ['runtime', 'test_sample_misclass', 'train_misclass', 'valid_sample_misclass', 'test_misclass', 
               'train_sample_misclass']

In [ ]:
#receptive field size
deep = 601
shallow = 609
linear =  600
deep_smac = 183
shallow_smac = 368



In [ ]:
df = load_data_frame('data/models/pytorch/auto-diag/final-smac-dense-fixed/')
df = df[df.finished == 1]
df = df.fillna('-')
df.runtime = pd.to_timedelta(np.round(df.runtime), unit='s')
df = df[(df.test_on_eval == 1) & (df.n_folds == 5)]
df = remove_columns_with_same_value(df)
print(len(df))
df = df.rename(columns=dict(shrink_the_spikes='shrink',
                            channel_standardize='chan_std',
                            low_cut_hz='low_hz',
                            max_abs_val='max_val',
                            high_cut_hz='high_hz',
                            moving_demean='m_mean',
                            moving_standardize='m_std')).drop(['train_loss', 'valid_loss',
                                                       'test_loss', 'valid_misclass', 'seed',
                                                       ], axis=1)
#df = df.drop(['save_predictions', 'n_chan_factor',
#             'n_start_chans', 'final_conv_length'], axis=1)

print(len(df))
df

In [ ]:
df = load_data_frame('data/models/pytorch/auto-diag/final-eval/')
df = df[df.finished == 1]
df = df.fillna('-')
df.runtime = pd.to_timedelta(np.round(df.runtime), unit='s')
df = df[(df.test_on_eval == 1) & (df.n_folds == 5)]
df = remove_columns_with_same_value(df)
print(len(df))
df = df.rename(columns=dict(shrink_the_spikes='shrink',
                            channel_standardize='chan_std',
                            low_cut_hz='low_hz',
                            max_abs_val='max_val',
                            high_cut_hz='high_hz',
                            moving_demean='m_mean',
                            moving_standardize='m_std')).drop(['train_loss', 'valid_loss',
                                                       'test_loss', 'valid_misclass', 'seed',
                                                       ], axis=1)
df = df.drop(['save_predictions', 'n_chan_factor',
             'n_start_chans', 'final_conv_length'], axis=1)

print(len(df))
df

In [ ]:
for key in ('test_misclass', 'test_sensitivity', 'test_specificity', 'test_sample_misclass'):
    val = np.mean(df[(df.model_name == 'deep')][key]) * 100
    if 'misclass' in key:
        val = 100 - val
    print(np.round(val,1))

In [ ]:
for key in ('test_misclass', 'test_sensitivity', 'test_specificity', 'test_sample_misclass'):
    val = np.mean(df[(df.model_name == 'shallow')][key]) * 100
    if 'misclass' in key:
        val = 100 - val
    print(np.round(val,1))

In [ ]:
for key in ('test_misclass', 'test_sensitivity', 'test_specificity', 'test_sample_misclass'):
    val = np.mean(df[(df.model_name == 'linear')][key]) * 100
    if 'misclass' in key:
        val = 100 - val
    print(np.round(val,1))

In [ ]:
reduced_df = df.copy()
for col in reduced_df.columns:
    if 'train_' in col or 'valid_' in col or 'auc_' in col:
        reduced_df = reduced_df.drop(col, axis=1)
        
reduced_df.test_sensitivity = np.round(reduced_df.test_sensitivity * 100, 1)
reduced_df.test_specificity = np.round(reduced_df.test_specificity * 100, 1)
reduced_df['test_accuracy'] = np.round(100 - reduced_df.test_misclass * 100, 1)
reduced_df['test_crop_accuracy'] = np.round(100 - reduced_df.test_sample_misclass * 100, 1) 
reduced_df = reduced_df.drop(['test_sample_misclass', 'test_misclass'], axis=1)
reduced_df

In [ ]:
df = load_data_frame('data/models/pytorch/auto-diag/6000-input//')
df = df[df.finished == 1]
df = df.fillna('-')
df.runtime = pd.to_timedelta(np.round(df.runtime), unit='s')
df = remove_columns_with_same_value(df)
print(len(df))
df = df.rename(columns=dict(shrink_the_spikes='shrink',
                            channel_standardize='chan_std',
                            low_cut_hz='low_hz',
                            max_abs_val='max_val',
                            high_cut_hz='high_hz',
                            moving_demean='m_mean',
                            moving_standardize='m_std')).drop(['train_loss', 'valid_loss',
                                                       'test_loss', 'valid_misclass',
                                                       ], axis=1)
df = df.drop(['n_chan_factor',
             'n_start_chans', 'final_conv_length'], axis=1)
df.test_misclass = df.test_misclass * 100

df['test_accuracy'] = 100 - df.test_misclass
print(len(df))
df

In [ ]:
some_df = np.load('data/models/pytorch/auto-diag/final-eval/4/before_stop_df.pkl')
full_df = np.load('data/models/pytorch/auto-diag/final-eval/4/epochs_df.pkl')

In [ ]:
plt.plot(some_df.train_misclass)
plt.plot(some_df.valid_misclass)
plt.plot(some_df.test_misclass)

In [ ]:
plt.plot(full_df.test_misclass)